In [2]:
import requests
import pandas as pd

df = pd.read_csv("list_regex_format_12bills.csv")
df.head(2)

,Unnamed: 0,Title,BillNumber,DocketNumber,GeneralCourtNumber,PrimarySponsor,Cosponsors,JointSponsor,BillHistory,LegislationTypeName,Pinslip,DocumentText,EmergencyPreamble,RollCalls,Attachments,CommitteeRecommendations,Amendments,List References
0,2299,An Act relative to collective bargaining right...,H3069,HD2435,193.0,"{'Id': 'PJK1', 'Name': 'Patrick Joseph Kearney...","[{'Id': 'PJK1', 'Name': 'Patrick Joseph Kearne...",NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,"By Representative Kearney of Scituate, a petit...",\tSECTION 1. Section 1 of Chapter 150E of the ...,NaN,[],[],[],[],"[['1', '150E'], ['23', '268A']]"
1,2358,An Act relative to the open meeting law,H3121,HD2204,193.0,"{'Id': 'E_U1', 'Name': 'Erika Uyterhoeven', 'T...","[{'Id': 'E_U1', 'Name': 'Erika Uyterhoeven', '...",NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,"By Representative Uyterhoeven of Somerville, a...",\tSECTION 18 of chapter 30A of the General Law...,NaN,[],[],[],[],"[['18', '30A']]"


List References: section number and chapter format

In [3]:
df['List References'] 

0                       [['1', '150E'], ['23', '268A']]
1                                       [['18', '30A']]
2                                                   NaN
3     [['2', '15D'], ['2', '18B'], ['11', '15D'], ['...
4                                       [['10', '70B']]
5                                        [['77', '54']]
6                          [['1', '125'], ['73', '7C']]
7                       [['1', '150E'], ['23', '268A']]
8                [['7', '4'], ['7', '4'], ['10', '66']]
9                                                   NaN
10    [['2', '15D'], ['2', '18B'], ['11', '15D'], ['...
11                                       [['77', '54']]
Name: List References, dtype: object

List References: section number and chapter format

In [4]:
df_cleaned = df[['Title', 'BillNumber', 'DocketNumber', 'DocumentText', 'List References']]
df_cleaned

,Title,BillNumber,DocketNumber,DocumentText,List References
0,An Act relative to collective bargaining right...,H3069,HD2435,\tSECTION 1. Section 1 of Chapter 150E of the ...,"[['1', '150E'], ['23', '268A']]"
1,An Act relative to the open meeting law,H3121,HD2204,\tSECTION 18 of chapter 30A of the General Law...,"[['18', '30A']]"
2,An Act relative to vehicle recalls,H400,HD1240,"\tSECTION 1. Chapter 90 of the General Laws, a...",NaN
3,An Act providing affordable and accessible hig...,H489,HD2794,\tSECTION 1. The seventh paragraph of section ...,"[['2', '15D'], ['2', '18B'], ['11', '15D'], ['..."
4,"An Act to Improve access, opportunity, and cap...",H538,HD485,\tSECTION 1. To create a new capital infrastru...,"[['10', '70B']]"
5,An Act providing a local option for ranked cho...,H711,HD1183,\tSECTION 1. Section 44A of chapter 43 of the ...,"[['77', '54']]"
6,An Act establishing a jail and prison construc...,S1979,SD661,\tSECTION 1. Chapter 7C of the General Laws is...,"[['1', '125'], ['73', '7C']]"
7,An Act relative to collective bargaining right...,S2014,SD1208,\tSECTION 1. Section 1 of chapter 150E of the ...,"[['1', '150E'], ['23', '268A']]"
8,An Act extending the public records law to the...,S2064,SD390,\tSECTION 1. Section 7 of chapter 4 of the Gen...,"[['7', '4'], ['7', '4'], ['10', '66']]"
9,An Act to end discriminatory outcomes in vocat...,S257,SD2312,\tSECTION 1. Chapter 74 of the General Laws is...,NaN


In [5]:
import ast 

new_df = df_cleaned.copy()

new_df['List References'] = df_cleaned['List References'].apply(lambda x: "None" if pd.isna(x) else ast.literal_eval(x))

In [6]:
new_df['List References']

0                       [[1, 150E], [23, 268A]]
1                                   [[18, 30A]]
2                                          None
3     [[2, 15D], [2, 18B], [11, 15D], [19, 66]]
4                                   [[10, 70B]]
5                                    [[77, 54]]
6                          [[1, 125], [73, 7C]]
7                       [[1, 150E], [23, 268A]]
8                    [[7, 4], [7, 4], [10, 66]]
9                                          None
10    [[2, 15D], [2, 18B], [11, 15D], [19, 66]]
11                                   [[77, 54]]
Name: List References, dtype: object

In [7]:
new_df['List References'].values[3]

[['2', '15D'], ['2', '18B'], ['11', '15D'], ['19', '66']]

In [8]:
import numpy as np

def make_api_call(item):
    result = """"""
    try:
        # unpack section and chapter for example: ['2', '15D']
        section, chapter = item
        r = requests.get(f'https://malegislature.gov/api/Chapters/{chapter}/Sections/{section}', verify=False)
        r = r.json()

        # fields to extract
        result =  r.get("Text", np.nan)
        return result
    except:
        pass

In [10]:
sample_1 = new_df['List References'].values[3]

result = """"""
# Storing and printing each pair
formatted_data = []
for pair in sample_1:
    string = make_api_call(pair)
    result += string
    

print(result)
print(len(result))

c:\Users\16178\anaconda3\envs\maple\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16178\anaconda3\envs\maple\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16178\anaconda3\envs\maple\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Section 2. There shall be in the executive office of education a department of early education and care, in this chapter called the department, which shall be the state agency responsible for compliance with early education and care services under the Personal Responsibility and Work Opportunity Reconciliation Act of 1996 (Pub. L. 104&ndash;193), or any successor federal statute. The department shall be the state education agency for the purposes of early education and care services under federal law. The department shall seek, apply for and encourage the use of any federal funds for early education and care services, and shall facilitate the coordination of federal, state, and local policies concerning early education and care. The department shall be under the supervision and management of the commissioner of early education and care, in this chapter called the commissioner.  

The department shall:&mdash;  

(a) be the lead agency of the commonwealth for administering and providing 

c:\Users\16178\anaconda3\envs\maple\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


this is in the format of section, chapter

In [11]:
new_df['MGL'] = new_df['List References'].apply(lambda pairs: [make_api_call(item) for item in pairs if item is not None])

c:\Users\16178\anaconda3\envs\maple\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16178\anaconda3\envs\maple\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16178\anaconda3\envs\maple\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16178\anac

In [32]:
new_df.iloc[3]

Title              An Act providing affordable and accessible hig...
BillNumber                                                      H489
DocketNumber                                                  HD2794
DocumentText       \tSECTION 1. The seventh paragraph of section ...
List References            [[2, 15D], [2, 18B], [11, 15D], [19, 66]]
MGL                [Section 2. There shall be in the executive of...
Name: 3, dtype: object

In [33]:
new_df['MGL'].values[3]

["Section 2. There shall be in the executive office of education a department of early education and care, in this chapter called the department, which shall be the state agency responsible for compliance with early education and care services under the Personal Responsibility and Work Opportunity Reconciliation Act of 1996 (Pub. L. 104&ndash;193), or any successor federal statute. The department shall be the state education agency for the purposes of early education and care services under federal law. The department shall seek, apply for and encourage the use of any federal funds for early education and care services, and shall facilitate the coordination of federal, state, and local policies concerning early education and care. The department shall be under the supervision and management of the commissioner of early education and care, in this chapter called the commissioner.  \r\n\r\nThe department shall:&mdash;  \r\n\r\n(a) be the lead agency of the commonwealth for administering 

In [40]:
new_df['combined_MGL'] = new_df['MGL'].apply(lambda lst: ' '.join([str(item) for item in lst if item is not None]))

In [43]:
new_df['length_MGL'] = new_df['combined_MGL'].apply(lambda x: len(x) if x else None)

In [44]:
new_df['length_MGL']

0     12170.0
1      4047.0
2         NaN
3     14010.0
4      5565.0
5       680.0
6      2813.0
7     12170.0
8     78184.0
9         NaN
10    14010.0
11      680.0
Name: length_MGL, dtype: float64

In [46]:
new_df.to_csv("12billswith.csv")